In [7]:
import pandas as pd 
import os 
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import  mean_squared_error, f1_score, accuracy_score, balanced_accuracy_score
from xgboost import XGBRegressor, XGBClassifier
from sklearn.preprocessing import KBinsDiscretizer
import numpy as np

In [8]:
os.getcwd()
os.listdir()

['.ipynb_checkpoints',
 'Archive',
 'data',
 'project.docx',
 'Submitted',
 'XGBoosting-Copy1.ipynb',
 'XGBoosting-Copy2.ipynb',
 'XGBoosting.ipynb']

In [9]:
scores = pd.read_csv("data/scores.csv", encoding="UTF-8")
# scores= scores[["msoa", "ses_asc"]]
scores.head()

,msoa,occupation_11,qualification_11,log_med_house_price_11,log_med_income_11,score_11,occupation_21,qualification_21,log_med_house_price_21,log_med_income_21,score_21,rank_11,rank_21,ses_asc,score_pr_11,score_pr_21,score_pr_asc
0,E02000001,0.808089,0.683642,13.227222,11.259583,-12.484865,0.838010,0.741819,13.609717,11.530775,-12.790197,3200.0,3259.0,-0.305333,1.871166,99.969325,98.098160
1,E02000002,0.294332,0.181927,12.305543,10.574416,-11.618879,0.345062,0.309106,12.748400,10.682835,-11.779756,1242.0,1447.0,-0.160877,61.932515,44.386503,-17.546012
2,E02000003,0.354646,0.234547,12.383504,10.787986,-11.848281,0.374761,0.340151,12.873905,10.837638,-11.948515,2053.0,2066.0,-0.100234,37.055215,63.374233,26.319018
3,E02000004,0.331928,0.178700,12.351755,10.735343,-11.784667,0.393706,0.307432,12.938443,10.859018,-11.972737,1843.0,2146.0,-0.188069,43.496933,65.828221,22.331288
4,E02000005,0.263819,0.175888,12.191133,10.707945,-11.736966,0.354646,0.337840,12.751303,10.781058,-11.881081,1679.0,1830.0,-0.144115,48.527607,56.134969,7.607362


In [10]:
c2011_percent = pd.read_csv("data/c2011_percent.csv", encoding="UTF-8")

c2011_percent = c2011_percent.drop(columns=[ 'TCITY15CD','MSOA11NM','Unnamed: 0'])
list(c2011_percent.columns)

['MSOA11CD',
 'TCITY15NM',
 'All usual residents',
 'Area (Hectares)',
 'Density (number of persons per hectare)',
 'Females',
 'Lives in a communal establishment',
 'Lives in a household',
 'Males',
 'Schoolchild or full-time student aged 4 and over at their non term-time address',
 'Age 0 to 4',
 'Age 10 to 14',
 'Age 15',
 'Age 16 to 17',
 'Age 18 to 19',
 'Age 20 to 24',
 'Age 25 to 29',
 'Age 30 to 44',
 'Age 45 to 59',
 'Age 5 to 7',
 'Age 60 to 64',
 'Age 65 to 74',
 'Age 75 to 84',
 'Age 8 to 9',
 'Age 85 to 89',
 'Age 90 and over',
 'Mean Age',
 'Median Age',
 'All usual residents aged 16+',
 'Divorced or formerly in a same-sex civil partnership which is now legally dissolved',
 'In a registered same-sex civil partnership',
 'Married',
 'Separated (but still legally married or still legally in a same-sex civil partnership)',
 'Single (never married or never registered a same-sex civil partnership)',
 'Widowed or surviving partner from a same-sex civil partnership',
 'All categ

In [11]:
combine_11 =pd.merge(c2011_percent, scores, left_on='MSOA11CD', right_on='msoa')
combine_11

,MSOA11CD,TCITY15NM,All usual residents,Area (Hectares),Density (number of persons per hectare),Females,Lives in a communal establishment,Lives in a household,Males,Schoolchild or full-time student aged 4 and over at their non term-time address,...,qualification_21,log_med_house_price_21,log_med_income_21,score_21,rank_11,rank_21,ses_asc,score_pr_11,score_pr_21,score_pr_asc
0,E02000053,London,8697.0,0.017825,56.1,0.491779,0.051397,0.948603,0.508221,0.015408,...,0.000000,13.017005,0.000000,-1.229094,53.0,59.0,9.761221,98.404908,1.809816,-96.595092
1,E02000209,London,8414.0,0.011919,83.9,0.512242,0.000357,0.999643,0.487758,0.010459,...,0.350992,12.765691,10.843514,-11.944034,2173.0,2043.0,-0.064273,33.374233,62.668712,29.294479
2,E02000054,London,7206.0,0.020819,48.0,0.507077,0.004302,0.995698,0.492923,0.015543,...,0.495579,13.022545,10.904138,-12.061959,2824.0,2453.0,0.059885,13.404908,75.245399,61.840491
3,E02000055,London,11130.0,0.012940,77.3,0.504582,0.021294,0.978706,0.495418,0.014645,...,0.462875,13.076845,10.853232,-12.008135,2611.0,2276.0,0.022807,19.938650,69.815951,49.877301
4,E02000056,London,9212.0,0.035031,28.5,0.531155,0.018454,0.981546,0.468845,0.026596,...,0.643790,14.093980,11.246365,-12.526347,3223.0,3204.0,0.021895,1.165644,98.282209,97.116564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255,E02003999,Carlisle,9380.0,0.920013,1.1,0.511194,0.019296,0.980704,0.488806,0.012687,...,0.318354,12.154785,10.714440,-11.778554,1612.0,1446.0,-0.055233,50.582822,44.355828,-6.226994
3256,E02005068,Chatham,7042.0,0.578613,1.7,0.504402,0.010650,0.989350,0.495598,0.019597,...,0.307692,12.959847,11.033292,-12.164703,2844.0,2744.0,-0.034355,12.791411,84.171779,71.380368
3257,E02004497,Chelmsford,7848.0,0.187663,5.3,0.510576,0.008792,0.991208,0.489424,0.018731,...,0.330693,12.977340,10.905974,-12.043129,2761.0,2395.0,0.049765,15.337423,73.466258,58.128834
3258,E02004493,Chelmsford,10127.0,0.030088,33.2,0.504098,0.014219,0.985781,0.495902,0.014516,...,0.399019,12.691584,10.851296,-11.973225,2688.0,2149.0,0.087955,17.576687,65.920245,48.343558


In [12]:
combine_11['score_change'] = combine_11['score_pr_21'] - combine_11['score_pr_11']
score_std_11 = combine_11['score_pr_11'].std()
combine_11['gentrified'] = np.where(combine_11['score_pr_11'] >= score_std_11, 1, 0)

y_train_11=combine_11[['gentrified']]
y_train_11

,gentrified
0,1
1,1
2,0
3,0
4,0
...,...
3255,1
3256,0
3257,0
3258,0


In [13]:
X_train_11 = combine_11.drop(['ses_asc', 'gentrified', 'msoa','MSOA11CD','TCITY15NM'], axis=1)
X_train_11[np.isinf(X_train_11)] = np.finfo(np.float32).max
X_train_11

,All usual residents,Area (Hectares),Density (number of persons per hectare),Females,Lives in a communal establishment,Lives in a household,Males,Schoolchild or full-time student aged 4 and over at their non term-time address,Age 0 to 4,Age 10 to 14,...,qualification_21,log_med_house_price_21,log_med_income_21,score_21,rank_11,rank_21,score_pr_11,score_pr_21,score_pr_asc,score_change
0,8697.0,0.017825,56.1,0.491779,0.051397,0.948603,0.508221,0.015408,0.077038,0.044383,...,0.000000,13.017005,0.000000,-1.229094,53.0,59.0,98.404908,1.809816,-96.595092,-96.595092
1,8414.0,0.011919,83.9,0.512242,0.000357,0.999643,0.487758,0.010459,0.094723,0.079986,...,0.350992,12.765691,10.843514,-11.944034,2173.0,2043.0,33.374233,62.668712,29.294479,29.294479
2,7206.0,0.020819,48.0,0.507077,0.004302,0.995698,0.492923,0.015543,0.069942,0.048293,...,0.495579,13.022545,10.904138,-12.061959,2824.0,2453.0,13.404908,75.245399,61.840491,61.840491
3,11130.0,0.012940,77.3,0.504582,0.021294,0.978706,0.495418,0.014645,0.071698,0.044205,...,0.462875,13.076845,10.853232,-12.008135,2611.0,2276.0,19.938650,69.815951,49.877301,49.877301
4,9212.0,0.035031,28.5,0.531155,0.018454,0.981546,0.468845,0.026596,0.072623,0.063830,...,0.643790,14.093980,11.246365,-12.526347,3223.0,3204.0,1.165644,98.282209,97.116564,97.116564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255,9380.0,0.920013,1.1,0.511194,0.019296,0.980704,0.488806,0.012687,0.044030,0.054478,...,0.318354,12.154785,10.714440,-11.778554,1612.0,1446.0,50.582822,44.355828,-6.226994,-6.226994
3256,7042.0,0.578613,1.7,0.504402,0.010650,0.989350,0.495598,0.019597,0.056944,0.066032,...,0.307692,12.959847,11.033292,-12.164703,2844.0,2744.0,12.791411,84.171779,71.380368,71.380368
3257,7848.0,0.187663,5.3,0.510576,0.008792,0.991208,0.489424,0.018731,0.045107,0.055046,...,0.330693,12.977340,10.905974,-12.043129,2761.0,2395.0,15.337423,73.466258,58.128834,58.128834
3258,10127.0,0.030088,33.2,0.504098,0.014219,0.985781,0.495902,0.014516,0.075047,0.050459,...,0.399019,12.691584,10.851296,-11.973225,2688.0,2149.0,17.576687,65.920245,48.343558,48.343558


In [14]:
X_train_11, X_test_11, y_train_11, y_test_11 = train_test_split(X_train_11, y_train_11, test_size=0.3, random_state=42)

model_11 = xgb.XGBClassifier()
model_11.fit(X_train_11, y_train_11)

y_pred_11 = model_11.predict(X_test_11)

In [15]:
mse_test_baseline_11 = mean_squared_error(y_test_11, y_pred_11)
f1_test_baseline_11 = f1_score(y_test_11, y_pred_11, average='weighted')
accuracy_test_baseline_11 = accuracy_score(y_test_11, y_pred_11)
balanced_accuracy_test_baseline_11 = balanced_accuracy_score(y_test_11, y_pred_11)

print("MSE (Test) 2011:", mse_test_baseline_11)
print("F1 Score (Test) 2011:", f1_test_baseline_11)
print("Accuracy (Test) 2011:", accuracy_test_baseline_11)
print("Balanced Accuracy (Test) 2011:", balanced_accuracy_test_baseline_11)

MSE (Test) 2011: 0.003067484662576687
F1 Score (Test) 2011: 0.9969338951620051
Accuracy (Test) 2011: 0.9969325153374233
Balanced Accuracy (Test) 2011: 0.9968715836326723


In [16]:
param_grid = {
    'learning_rate': [0.1, 0.01],
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200, 300]}

In [17]:
cv_model_11 = GridSearchCV(estimator=XGBRegressor(random_state=42), param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
cv_model_11.fit(X_train_11, y_train_11)

optimal_params_11 = cv_model_11.best_params_

optimal_model_11 = xgb.XGBClassifier(random_state=42, **optimal_params_11)
optimal_model_11.fit(X_train_11, y_train_11)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='binary:logistic', ...)

In [20]:

y_test_pred_optimal_11 = optimal_model_11.predict(X_test_11)

mse_test_optimal_11 = mean_squared_error(y_test_11, y_test_pred_optimal_11)
f1_test_optimal_11 = f1_score(y_test_11, y_test_pred_optimal_11, average='weighted')
accuracy_test_optimal_11 = accuracy_score(y_test_11, y_test_pred_optimal_11)
balanced_accuracy_test_optimal_11 = balanced_accuracy_score(y_test_11, y_test_pred_optimal_11)

print("MSE: (2011)" , mse_test_optimal_11)
print("F1 Score: (2011)", f1_test_optimal_11)
print("Accuracy: (2011)", accuracy_test_optimal_11)
print("Balanced Accuracy (2011):", balanced_accuracy_test_optimal_11)

MSE: (2011) 0.003067484662576687
F1 Score: (2011) 0.9969338951620051
Accuracy: (2011) 0.9969325153374233
Balanced Accuracy (2011): 0.9968715836326723


In [21]:
c2021_percent = pd.read_csv("data/c2021_percent.csv", encoding="UTF-8")

c2021_percent = c2021_percent.drop(columns=['TCITY15CD','MSOA11NM','Unnamed: 0'])
combine_21 =pd.merge(c2021_percent, scores, left_on='MSOA11CD', right_on='msoa')

# combine_21['score_change'] = combine_21['score_pr_21'] - combine_21['score_pr_21']
score_std_21 = combine_21['score_pr_21'].std()
combine_21['gentrified'] = np.where(combine_21['score_pr_21'] >= score_std_21, 1, 0)

y_train_21=combine_21[['gentrified']]

X_train_21 = combine_21.drop(['ses_asc', 'gentrified', 'msoa','MSOA21CD','TCITY15NM','MSOA11CD'], axis=1)
X_train_21[np.isinf(X_train_21)] = np.finfo(np.float32).max
X_train_21

X_train_21, X_test_21, y_train_21, y_test_21 = train_test_split(X_train_21, y_train_21, test_size=0.3, random_state=42)

model_21 = xgb.XGBClassifier()
model_21.fit(X_train_21, y_train_21)

y_pred_21 = model_21.predict(X_test_21)

mse_test_baseline_21 = mean_squared_error(y_test_21, y_pred_21)
f1_test_baseline_21 = f1_score(y_test_21, y_pred_21, average='weighted')
accuracy_test_baseline_21 = accuracy_score(y_test_21, y_pred_21)
balanced_accuracy_test_baseline_21 = balanced_accuracy_score(y_test_21, y_pred_21)

print("MSE (Test) 2021:", mse_test_baseline_21)
print("F1 Score (Test) 2021:", f1_test_baseline_21)
print("Accuracy (Test) 2021:", accuracy_test_baseline_21)
print("Balanced Accuracy (Test) 2021:", balanced_accuracy_test_baseline_21)

param_grid = {
    'learning_rate': [0.1, 0.01],
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200, 300]}

cv_model_21 = GridSearchCV(estimator=XGBRegressor(random_state=42), param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
cv_model_21.fit(X_train_21, y_train_21)

optimal_params_21 = cv_model_21.best_params_

optimal_model_21 = xgb.XGBClassifier(random_state=42, **optimal_params_21)
optimal_model_21.fit(X_train_21, y_train_21)

y_test_pred_optimal_21 = optimal_model_21.predict(X_test_21)

mse_test_optimal_21 = mean_squared_error(y_test_21, y_test_pred_optimal_21)
f1_test_optimal_21 = f1_score(y_test_21, y_test_pred_optimal_21, average='weighted')
accuracy_test_optimal_21 = accuracy_score(y_test_21, y_test_pred_optimal_21)
balanced_accuracy_test_optimal_21 = balanced_accuracy_score(y_test_21, y_test_pred_optimal_21)

print("MSE (2021):", mse_test_optimal_21)
print("F1 Score (2021):", f1_test_optimal_21)
print("Accuracy (2021):", accuracy_test_optimal_21)
print("Balanced Accuracy (2021):", balanced_accuracy_test_optimal_21)


MSE (Test) 2021: 0.00408997955010225
F1 Score (Test) 2021: 0.9959201234489852
Accuracy (Test) 2021: 0.9959100204498977
Balanced Accuracy (Test) 2021: 0.9972260748959778
MSE (2021): 0.00408997955010225
F1 Score (2021): 0.9959201234489852
Accuracy (2021): 0.9959100204498977
Balanced Accuracy (2021): 0.9972260748959778
